In [2]:
#import tensorflow as tf
import os

import numpy as np
import pandas as pd
import re
import scipy as sp
from scipy.sparse import hstack, vstack
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

import sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

import pickle
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_union

In [3]:
def save_variable_pickle(var, name, model = False):
    filename = name + '.pickle'
    if model:
        joblib.dump(var, filename = filename)
    else:  
        with open(filename, 'wb') as f:
            pickle.dump(var, f, pickle.HIGHEST_PROTOCOL)

def load_variable_pickle(name, model = False):
    filename = name
    if model:
        return joblib.load(filename = filename)
    else:
        with open(filename, 'rb') as f:
            return pickle.load(f)
        

In [4]:
def choose_system(os):
    if os == 'mac':
        train_path = '/Users/yichen/Desktop/toxic/input/train.csv'
        test_path = '/Users/yichen/Desktop/toxic/input/test.csv'
    elif os == 'linux':
        train_path = '/home/yichen/Desktop/toxic/input/train.csv'
        test_path = '/home/yichen/Desktop/toxic/input/test.csv'
    elif os == 'aws':
        train_path = 'train.csv'
        test_path = 'test.csv'
    
    train = pd.read_csv(train_path).fillna(' ')
    test = pd.read_csv(test_path).fillna(' ')
    return train, test
    
    

In [5]:
def check_accuracy(true, estimate):
    acc = accuracy_score(true, estimate)
    logloss = log_loss(true, estimate)
    auc = roc_auc_score(true, esimate)
    print("accuracy score: {}, logloss: {}, auc: {}".format(acc, logloss, auc))

In [6]:
train_features.shape

NameError: name 'train_features' is not defined

In [7]:
train, test = choose_system('linux')
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
test.shape

(153164, 2)

In [8]:
repl = {
    "&lt;3": " good ", ":d": " good ",
    ":dd": " good ",
    ":p": " good ",
    "8)": " good ",
    ":-)": " good ",
    ":)": " good ",
    ";)": " good ",
    "(-:": " good ",
    "(:": " good ",
    "yay!": " good ",
    "yay": " good ",
    "yaay": " good ",
    "yaaay": " good ",
    "yaaaay": " good ",
    "yaaaaay": " good ",
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " bad ",
    ":(": " bad ",
    ":s": " bad ",
    ":-s": " bad ",
    "&lt;3": " heart ",
    ":d": " smile ",
    ":p": " smile ",
    ":dd": " smile ",
    "8)": " smile ",
    ":-)": " smile ",
    ":)": " smile ",
    ";)": " smile ",
    "(-:": " smile ",
    "(:": " smile ",
    ":/": " worry ",
    ":&gt;": " angry ",
    ":')": " sad ",
    ":-(": " sad ",
    ":(": " sad ",
    ":s": " sad ",
    ":-s": " sad ",
    r"\br\b": "are",
    r"\bu\b": "you",
    r"\bhaha\b": "ha",
    r"\bhahaha\b": "ha",
    r"\bdon't\b": "do not",
    r"\bdoesn't\b": "does not",
    r"\bdidn't\b": "did not",
    r"\bhasn't\b": "has not",
    r"\bhaven't\b": "have not",
    r"\bhadn't\b": "had not",
    r"\bwon't\b": "will not",
    r"\bwouldn't\b": "would not",
    r"\bcan't\b": "can not",
    r"\bcannot\b": "can not",
    r"\bi'm\b": "i am",
    "m": "am",
    "r": "are",
    "u": "you",
    "haha": "ha",
    "hahaha": "ha",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "can't": "can not",
    "cannot": "can not",
    "i'm": "i am",
    "m": "am",
    "i'll" : "i will",
    "its" : "it is",
    "it's" : "it is",
    "'s" : " is",
    "that's" : "that is",
    "weren't" : "were not",
}

In [9]:
keys = [i for i in repl.keys()]

new_train_data = []
new_test_data = []
ltr = train["comment_text"].tolist()
keys = [i for i in repl.keys()]

new_train_data = []
new_test_data = []
ltr = train["comment_text"].tolist()
lte = test["comment_text"].tolist()
for i in ltr:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_train_data.append(xx)
for i in lte:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_test_data.append(xx)
train["new_comment_text"] = new_train_data
test["new_comment_text"] = new_test_data

trate = train["new_comment_text"].tolist()
tete = test["new_comment_text"].tolist()
for i, c in enumerate(trate):
    trate[i] = re.sub('[^a-zA-Z ?!]+', '', str(trate[i]).lower())
for i, c in enumerate(tete):
    tete[i] = re.sub('[^a-zA-Z ?!]+', '', tete[i])
train["comment_text"] = trate
test["comment_text"] = tete
del trate, tete
train.drop(["new_comment_text"], axis=1, inplace=True)
test.drop(["new_comment_text"], axis=1, inplace=True)

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [10]:
all_text.shape

(312735,)

In [9]:
word_vectorizer = TfidfVectorizer(
    strip_accents='unicode',
    analyzer='word',
    min_df = 2,
    stop_words = 'english',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 3),
    max_features=30000
    )
    

char_vectorizer = TfidfVectorizer(
    strip_accents='unicode',
    analyzer='char',
    min_df = 2,
    ngram_range=(2, 6),
    max_features=30000)

vectorizer = make_union(word_vectorizer, char_vectorizer)

In [10]:
vectorizer.fit(all_text)
train_features = vectorizer.transform(train_text)
train_features.shape

(159571, 60000)

In [11]:
test_features = vectorizer.transform(test_text)

In [12]:
test_features.shape

(153164, 60000)

In [13]:
import pickle
from sklearn.externals import joblib
save_variable_pickle(train_features, 'train_features_save')
save_variable_pickle(test_features, 'test_features_save')

In [11]:
import pickle
from sklearn.externals import joblib
test_features = load_variable_pickle('test_features_save.pickle')
train_features = load_variable_pickle('train_features_save.pickle')
#clf = load_variable_pickle('_logistic_model5.pickle')

In [12]:
print(test_features.shape, train_features.shape)


(153164, 60000) (159571, 60000)


In [13]:
gbm = lgb.LGBMClassifier(objective = 'binary',
                        metric = 'auc',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=200)
#gbm.fit(meta_train_toxic_prediction, meta_train_toxic_valid,
 #       eval_metric='auc')

log_model = LogisticRegression(solver='sag')
svc_model = LinearSVC(random_state=1023)
type(log_model)
os.path.exists('/home/yichen/Desktop/toxic/_logistic_model.pickle')

True

In [14]:
def cv_split(train, target, K, random_state ,model):
    acc = []
    row = train.shape[0]
    if row != target.shape[0]:
        raise ValueError("At least one array required as input")
    num_per_fold = np.int(np.ceil(train.shape[0]/K))
    x_train, x_fold, y_train, y_fold1 = train_test_split(train, target, test_size = num_per_fold, random_state = random_state)
    model.fit(x_train, y_train)
    pred = model.predict(x_fold)
    acc.append(roc_auc_score(y_fold1, pred))
    #print("accuracy is {}".format(accuracy_score(y_fold1, pred)))
    i = 1
    if train.size != row:
        while i < K:
            if i != K - 1:
                x_fold = train[num_per_fold * i: num_per_fold * (i + 1), :]
                y_fold = target[num_per_fold * i: num_per_fold * (i + 1)]
                x_train = vstack([train[0: num_per_fold * i, : ], train[num_per_fold * (i + 1): , :]])
                y_train = pd.concat([target[0 : num_per_fold * i], target[num_per_fold * (i + 1): ]])
                #print("x_train: {}, y_train: {}, x_fold: {}, y_fold: {}".format(x_train.shape, y_train.shape, x_fold.shape, y_fold.shape))
            elif i == K-1:
                x_fold = train[num_per_fold * i: , :]
                y_fold = target[num_per_fold * i: ]
                x_train = train[0:num_per_fold * i, :]
                y_train = target[0:num_per_fold * i]
                
            
            y_fold1 = pd.concat([y_fold1[:], y_fold[:]])
            model.fit(x_train, y_train)
            pred1 = model.predict(x_fold)
            acc.append(roc_auc_score(y_fold, pred1))
            #rint("accuracy is {}".format(accuracy_score(y_fold, pred1)))
            pred = np.concatenate((pred, pred1), axis = 0)
            i = i + 1
    print("average roc_auc_score is: {}  for model type: {} ".format(np.mean(acc), type(model)))
    return pred# pred_all_testep

In [34]:
base_level_models = []
scores_auc = []
#submission = pd.DataFrame.from_dict({'id': test['id']})
y_fold_validation = load_variable_pickle('y_fold_validation.pickle')
#submission[class_name] = classifier.predict_proba(test_features)[:, 1]
MNB = MultinomialNB()

neigh2 = KNeighborsClassifier(n_neighbors=2)
neigh4 = KNeighborsClassifier(n_neighbors=4)
neigh8 = KNeighborsClassifier(n_neighbors=8)
neigh16 = KNeighborsClassifier(n_neighbors=16)
neigh32 = KNeighborsClassifier(n_neighbors=32)
neigh64 = KNeighborsClassifier(n_neighbors=64)
neigh128 = KNeighborsClassifier(n_neighbors=128)
neigh256 = KNeighborsClassifier(n_neighbors=256)
neigh512 = KNeighborsClassifier(n_neighbors=512)
neigh1024 = KNeighborsClassifier(n_neighbors=1024)

etc_levelone = ExtraTreesClassifier(n_estimators= 50, verbose = True, random_state=11)
ada = AdaBoostClassifier(n_estimators=100) #verbose = True)

rfc= RandomForestClassifier(n_estimators= 5, random_state=11, verbose = True)
#gdc = GradientBoostingClassifier(n_estimators=100, validation_fraction=0.2,
#                                 tol = 0.01, no_iter_no_change = 5,learning_rate=0.1, random_state=1023)

lgb_lvl1 = gbm = lgb.LGBMClassifier(objective = 'binary',
                        metric = 'auc',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=150) #device = 'gpu', gpu_use_dp = true)

"<class 'sklearn.svm.classes.LinearSVC'>"

In [35]:
name1 = '/home/yichen/Desktop/toxic/base_model/' + '_' + 'toxic' + '_' + str(type(svc_model)) + 'cv'+ '_'+ '.pickle'
name1

"/home/yichen/Desktop/toxic/base_model/_toxic_<class 'sklearn.svm.classes.LinearSVC'>cv_.pickle"

In [36]:
os.path.exists(name1)

False

In [37]:
def ready_for_leveltwo(models, train_data, test_data, _train_target, meta_model):
    #os_load_file_path = '/home/yichen/Desktop/toxic/base_model/'
    submission = pd.DataFrame.from_dict({'id': test['id']})
    for class_name in class_names:
        os_load_file_path = '/home/yichen/Desktop/toxic/base_model/'
        os_load_file_path = os_load_file_path + str(class_name) + '/'
        print(os_load_file_path)
        count = 0
        class_counter = 0
        train_target = train[class_name]
        for model in models:
            name = os_load_file_path + '_' + class_name + '_' + str(type(model)) + 'cv'+ '_'+ '.pickle'
            if count == 0:
                if os.path.exists(name):
                    with open(name, 'rb') as f:
                        meta_train = pickle.load(f)
                    print("old prediction stored: {}, shape: {}".format(type(model), meta_train.shape))
                else:
                    meta_train = cv_split(train = train_data, target=train_target, K = 5, random_state=1023, model = model)
                    with open(name, 'wb') as f:
                        pickle.dump(meta_train, f, pickle.HIGHEST_PROTOCOL)
                count = count + 1
            else:
                if os.path.exists(name):
                    with open(name, 'rb') as f:
                        temp = pickle.load(f)
                    print("old prediction stored: {}, shape: {}".format(type(model), temp.shape))
                else:
                    temp = cv_split(train = train_data, target=train_target, K = 5, random_state=1023, model = model)
                    with open(name, 'wb') as f:
                        pickle.dump(temp, f, pickle.HIGHEST_PROTOCOL)
                meta_train = np.column_stack((meta_train, temp))
                count = count + 1
                
        count = 0                
        print("cv finished")
        for model in models:
            name = os_load_file_path + '_' + class_name + '_' + str(type(model)) + 'base_lvl' + '_' + '.pickle'
            if count == 0:
                if os.path.exists(name):
                    with open(name, 'rb') as f:
                        meta_test = pickle.load(f)
                    print("old prediction stored: {}, shape: {}".format(type(model), meta_train.shape))
                else:
                    model.fit(train_data, train_target)
                    meta_test = model.predict(test_data)
                    with open(name, 'wb') as f:
                        pickle.dump(meta_test, f, pickle.HIGHEST_PROTOCOL)
                    print("model finish predication: {}".format(str(type(model))))
                count = count + 1
            else:
                if os.path.exists(name):
                    with open(name, 'rb') as f:
                        temp = pickle.load(f)
                    print("old prediction stored: {}, shape: {}".format(type(model), meta_train.shape))
                else:
                    model.fit(train_data, train_target)
                    temp = model.predict(test_data)
                    with open(name, 'wb') as f:
                        pickle.dump(temp, f, pickle.HIGHEST_PROTOCOL)
                    print("model finish predication: {}".format(str(type(model))))
                meta_test = np.column_stack((meta_test, temp))
                count = count + 1
            
        
        meta_train_valid = y_fold_validation[:, class_counter]
        print("meta_train: {}, y_fold: {}, meta_test: {}".format(meta_train.shape, meta_train_valid.shape, meta_test.shape))
        meta_model.fit(meta_train, meta_train_valid) 
        a = meta_model.predict(meta_test)
        sub_name = 'submission' + str(class_name)
        save_variable_pickle(a, sub_name)
        print(a.shape)
        submission[class_name] = a
        class_counter = class_counter + 1
        print("round: ")
    submission.to_csv('submission_final.csv', index=False)
    return submission

In [38]:
y_fold_validation[:, 0].shape

(159571,)

In [39]:
from sklearn import svm

nusvc = svm.NuSVC(nu=0.01,random_state = 1023, verbose = True)
rfc= RandomForestClassifier(n_estimators= 50, random_state=11, verbose = True)

gdc = GradientBoostingClassifier(n_estimators=400, 
 learning_rate = 0.1, max_depth=5, random_state=1023, verbose = True)
lgb_lvl1 = lgb.LGBMClassifier(objective = 'binary',
                        metric = 'auc',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=150) #device = 'gpu', gpu_use_dp = true)
gbm = lgb.LGBMClassifier(objective = 'binary',
                        metric = 'auc',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=1000)
model_set1 = []
model_set1.append(etc_levelone)
model_set1.append(ada)
model_set1.append(svc_model)
model_set1.append(log_model)
model_set1.append(MNB)
model_set1.append(lgb_lvl1)
model_set1.append(rfc)
model_set1.append(gdc)
model_set1.append(nusvc)
#model_set1.append(neigh2)
#model_set1.append(neigh4)
#model_set1.append(neigh8)
#model_set1.append(neigh16)
#model_set1.append(neigh32)
#model_set1.append(neigh64)

In [ ]:
meta_train1 = ready_for_leveltwo(model_set1, train_features, test_features, y_fold_validation, meta_model=gbm)
meta_train1.shape

/home/yichen/Desktop/toxic/base_model/toxic/
old prediction stored: <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>, shape: (159571,)
old prediction stored: <class 'sklearn.ensemble.weight_boosting.AdaBoostClassifier'>, shape: (159571,)
old prediction stored: <class 'sklearn.svm.classes.LinearSVC'>, shape: (159571,)
old prediction stored: <class 'sklearn.linear_model.logistic.LogisticRegression'>, shape: (159571,)
old prediction stored: <class 'sklearn.naive_bayes.MultinomialNB'>, shape: (159571,)
old prediction stored: <class 'lightgbm.sklearn.LGBMClassifier'>, shape: (159571,)
old prediction stored: <class 'sklearn.ensemble.forest.RandomForestClassifier'>, shape: (159571,)
      Iter       Train Loss   Remaining Time 
         1           0.5605          354.92m


In [ ]:
model_set1.append(neigh2)
model_set1.append(neigh4)
model_set1.append(neigh8)
model_set1.append(neigh16)
model_set1.append(neigh32)
meta_train2 = ready_for_leveltwo(model_set1, train_features, test_features, y_fold_validation, meta_model=gbm)

In [24]:
def cv_split(train, target, K, random_state ,model):
    acc = []
    row = train.shape[0]
    if row != target.shape[0]:
        raise ValueError("At least one array required as input")
    num_per_fold = np.int(np.ceil(train.shape[0]/K))
    x_train, x_fold, y_train, y_fold1 = train_test_split(train, target, test_size = num_per_fold, random_state = random_state)
    model.fit(x_train, y_train)
    pred = model.predict(x_fold)
    acc.append(roc_auc_score(y_fold1, pred))
    #print("accuracy is {}".format(accuracy_score(y_fold1, pred)))
    i = 1
    if train.size != row:
        while i < K:
            if i != K - 1:
                x_fold = train[num_per_fold * i: num_per_fold * (i + 1), :]
                y_fold = target[num_per_fold * i: num_per_fold * (i + 1)]
                x_train = vstack([train[0: num_per_fold * i, : ], train[num_per_fold * (i + 1): , :]])
                y_train = pd.concat([target[0 : num_per_fold * i], target[num_per_fold * (i + 1): ]])
                #print("x_train: {}, y_train: {}, x_fold: {}, y_fold: {}".format(x_train.shape, y_train.shape, x_fold.shape, y_fold.shape))
            elif i == K-1:
                x_fold = train[num_per_fold * i: , :]
                y_fold = target[num_per_fold * i: ]
                x_train = train[0:num_per_fold * i, :]
                y_train = target[0:num_per_fold * i]
                
            
            y_fold1 = pd.concat([y_fold1[:], y_fold[:]])
            model.fit(x_train, y_train)
            pred1 = model.predict(x_fold)
            acc.append(roc_auc_score(y_fold, pred1))
            #rint("accuracy is {}".format(accuracy_score(y_fold, pred1)))
            pred = np.concatenate((pred, pred1), axis = 0)
            i = i + 1
    print("average roc_auc_score is: {}  for model type: {} ".format(np.mean(acc), type(model)))
    return pred# pred_all_testep

In [52]:
def single_class_cv_mutiple_time(model):
    result = np.arange(159571*6).reshape(159571,6)
    #all_test_result = np.arange(153164*6).reshape(153164,6)
    col = 0
    for class_name in class_names:
        train_target = train[class_name]
        test = test_features
        pred1, overall_test_result= cv_split(train = train_features, target = train_target, K = 5, random_state=1023, model=model)
        result[:, col] = pred1
        all_test_result[:, col] = overall_test_result
        #y_fold_result[:, col] = y_fold
        col = col + 1
        print("pred shape: {} and overall_test shape: {} ".format(pred1.shape, all_test_result.shape))
    
    return result, all_test_result


In [29]:
def param_tune(model, train, train_t):
    score_auc = []
    for class_name in class_names:
        train_target = train_t[class_name]
        cv_score = np.mean(cross_val_score(
            model, train, train_target, cv=3, scoring='roc_auc'))
    scores_auc.append(np.abs(cv_score))
    print('auc CV score for class {} is {}'.format(class_name, np.abs(cv_score)))

In [32]:
gdc5 = GradientBoostingClassifier(n_estimators=400, 
 learning_rate = 0.1, max_depth=5, random_state=1023, verbose = True)

In [33]:
param_tune(gdc, train_features, train)

      Iter       Train Loss   Remaining Time 
         1           0.5875           15.64m
         2           0.5672           14.09m
         3           0.5556           13.42m
         4           0.5455           13.02m
         5           0.5377           12.74m
         6           0.5318           12.54m
         7           0.5272           12.33m


KeyboardInterrupt: 

In [139]:
scores_auc = []
submission = pd.DataFrame.from_dict({'id': test['id']})
#classifier = LogisticRegression()
#for class_name in class_names:
train_target = train['toxic']
    #print(train_target.shape)
classifier = LogisticRegression(solver='sag')

    #cv_score = np.mean(cross_val_score(
     #   classifier, train_features, train_target, cv=3, scoring='neg_log_loss'))
    #scores_auc.append(np.abs(cv_score))
    #print('auc CV score for class {} is {}'.format(class_name, np.abs(cv_score)))

classifier.fit(train_features, train_target)
a = classifier.predict_proba(test_features)
print(a.shape)
a
submission[class_name] = a[:, 1]
    #save_varible_pickle(classifier, _logistic, model = True)
#print('Total auc CV score is {}'.format(np.mean(scores_auc)))

(153164, 2)


In [18]:
submission.to_csv('submission.csv', index=False)